# VAE
Variational autoencoder [1] models inherit autoencoder architecture, but make strong assumptions concerning the distribution of latent variables. They use variational approach for latent representation learning

[1] Auto-Encoding Variational Bayes, Diederik P Kingma, Max Welling 2013

In [1]:
# Setup
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import os
from tensorflow.examples.tutorials.mnist import input_data
from tensorflow.contrib.slim import fully_connected as fc

/anaconda/envs/py35/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# Load MNIST data
mnist = input_data.read_data_sets('../../MNIST_data', one_hot=True)
X_dim = mnist.train.images.shape[1]
y_dim = mnist.train.labels.shape[1]
num_data = mnist.train.num_examples

def plot(samples):
    fig = plt.figure(figsize=(4, 4))
    gs = gridspec.GridSpec(4, 4)
    gs.update(wspace=0.05, hspace=0.05)

    for i, sample in enumerate(samples):
        ax = plt.subplot(gs[i])
        plt.axis('off')
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.set_aspect('equal')
        plt.imshow(sample.reshape(28, 28), cmap='Greys_r')

    return fig

Extracting ../../MNIST_data/train-images-idx3-ubyte.gz
Extracting ../../MNIST_data/train-labels-idx1-ubyte.gz
Extracting ../../MNIST_data/t10k-images-idx3-ubyte.gz
Extracting ../../MNIST_data/t10k-labels-idx1-ubyte.gz


In [127]:
# Hyperparamters. Your job to find these.
num_epochs = 201
batch_size = 50
z_dim = 20
lr = 1e-4

# Encoder 
Implement the encoder network $Q_\phi(z \mid x)$. Use Tensorflow's `fully_connected` function [API guide](https://www.tensorflow.org/api_docs/python/tf/contrib/layers/fully_connected) to write less lines or code. Use 3 shared FC layers and 1 FC layer at the end for the mean $\mu(X_i)$ and another 1 FC layer at the end for the diagonal variance $\sigma(X_i)$.

Also, implement the reparameterization trick we talked about in the class.

- Do take a look at the API and make sure to not use any non-linearities at the final layer!
- Make it return log of the variance so we work with smaller numbers (for stability)!

In [128]:
X = tf.placeholder(tf.float32, shape=[None, X_dim])

def Q(X):
    fc1 = tf.contrib.layers.fully_connected(X, 128)
    fc2 = tf.contrib.layers.fully_connected(fc1, 128)
    fc3 = tf.contrib.layers.fully_connected(fc2, 128)
    z_mu = tf.contrib.layers.fully_connected(fc3, z_dim, activation_fn = None)
    z_logvar = tf.contrib.layers.fully_connected(fc3, z_dim, activation_fn = None)
    return z_mu, z_logvar
    
def sample_z(mu, log_var):
    param = tf.random_normal(tf.shape(log_var))
    reparameterized_sample = mu + tf.exp(0.5 * log_var) * param
    return reparameterized_sample

# Decoder
Implement the decoder network $P_\theta(X \mid z)$. Use Tensorflow's `fully_connected` function [API guide](https://www.tensorflow.org/api_docs/python/tf/contrib/layers/fully_connected) to write less lines or code as well.

In this exercise, we will use Bernoulli MLP decoder explained in Appendix C.1 in the original paper.

Use 3 FC layers and 1 FC layer at the end for the unnormalized and normalized logits.

In [129]:
def P(z):
    fc4 = tf.contrib.layers.fully_connected(z, 128)
    fc5 = tf.contrib.layers.fully_connected(fc4, 128)
    fc6 = tf.contrib.layers.fully_connected(fc5, 128, activation_fn = tf.nn.tanh)
    logits = tf.contrib.layers.fully_connected(fc6, 784, activation_fn = None)
    prob = tf.nn.sigmoid(logits)
    return prob, logits

# Rest of the graph
We can define the rest of the network as below.

In [130]:
z_mu, z_logvar = Q(X)
z_sample = sample_z(z_mu, z_logvar)
X_samples, logits = P(z_sample)

# Graph loss function
You might find `tf.nn.sigmoid_cross_entropy_with_logits` function [API guide](https://www.tensorflow.org/api_docs/python/tf/nn/sigmoid_cross_entropy_with_logits) useful.

In [131]:
entropy = tf.reduce_sum(tf.nn.sigmoid_cross_entropy_with_logits(labels=X, logits=logits), axis=1)
recon_loss = tf.reduce_mean(entropy)

kl = -0.5 * tf.reduce_sum(z_logvar - (z_mu ** 2) - (tf.exp(z_logvar)) + 1, axis=1)
kl_loss = tf.reduce_mean(kl)

# Total loss
vae_loss = recon_loss + kl_loss

In [134]:
# Optimizer and solver
solver = tf.train.AdamOptimizer(learning_rate=lr).minimize(vae_loss)
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

# Train a _great_ generative model on MNIST!

Now, it's your job to experiment with hyperparameters defined in the beggining. The code below will also generate images and save the results to `figures/` directory. We are attaching the example images from our run and we expect your images to be of similar quality. 
![](out/080.png)


In [135]:
if not os.path.exists('figures/'):
    os.makedirs('figures/')

for epoch in range(num_epochs):
    for it in range(num_data // batch_size):
        X_mb, _ = mnist.train.next_batch(batch_size)

        _, loss = sess.run([solver, vae_loss], feed_dict={X: X_mb})

    if epoch % 5 == 0:
        print('Epoch: {}, Loss: {:.4}'. format(epoch, loss))

        samples = sess.run(X_samples, feed_dict={z_sample: np.random.randn(16, z_dim)})

        fig = plot(samples)
        plt.savefig('figures/{}.png'.format(str(epoch).zfill(3)), bbox_inches='tight')
        plt.close(fig)

Epoch: 0, Loss: 196.4
Epoch: 5, Loss: 187.2
Epoch: 10, Loss: 178.5
Epoch: 15, Loss: 154.7
Epoch: 20, Loss: 148.0
Epoch: 25, Loss: 145.3
Epoch: 30, Loss: 148.1
Epoch: 35, Loss: 149.2
Epoch: 40, Loss: 144.9
Epoch: 45, Loss: 148.1
Epoch: 50, Loss: 140.8
Epoch: 55, Loss: 128.0
Epoch: 60, Loss: 138.6
Epoch: 65, Loss: 134.6
Epoch: 70, Loss: 126.2
Epoch: 75, Loss: 134.1
Epoch: 80, Loss: 123.5
Epoch: 85, Loss: 125.0
Epoch: 90, Loss: 131.3
Epoch: 95, Loss: 127.1
Epoch: 100, Loss: 127.0
Epoch: 105, Loss: 115.4
Epoch: 110, Loss: 125.5
Epoch: 115, Loss: 124.5
Epoch: 120, Loss: 121.5
Epoch: 125, Loss: 119.6
Epoch: 130, Loss: 123.8
Epoch: 135, Loss: 124.0
Epoch: 140, Loss: 125.3
Epoch: 145, Loss: 123.5
Epoch: 150, Loss: 113.1
Epoch: 155, Loss: 118.6
Epoch: 160, Loss: 117.6
Epoch: 165, Loss: 110.6
Epoch: 170, Loss: 116.8
Epoch: 175, Loss: 111.2
Epoch: 180, Loss: 117.5
Epoch: 185, Loss: 110.4
Epoch: 190, Loss: 127.8
Epoch: 195, Loss: 121.1
Epoch: 200, Loss: 118.1
